In [6]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import pandas as pd
import json
import numpy as np
import requests

try:
    HERE = Path(__file__).resolve()
except NameError:
    HERE = Path.cwd()

In [7]:
# test annotations generated by prompt 1
data = pd.read_csv("gemma_annotated_prompt1.csv", dtype=str)
data.head(10)

,Unnamed: 0.1,Unnamed: 0,ingredient,amount,ingr_annotation,amount_annotation,norm_value,norm_unit,nutrition,ingr_json,amount_json
0,0,15,bio hähnchenschlegel,4 stk.,"{""anzahl"": 4, ""einheit"": ""stk.""}","{""zutat"": ""H\u00e4hnchenschlegel"", ""eigenschaf...",4.0,stück,NaN,"[{'ingredient': 'Hähnchenschlegel', 'propertie...","[{'amount': 4, 'unit': 'Stück'}]"
1,1,31,schokoriegel,1 stk.,"{""anzahl"": 1, ""einheit"": ""stk.""}","{""zutat"": ""Schokoriegel""}",1.0,stück,NaN,[{'ingredient': 'Schokoriegel'}],"[{'amount': 1, 'unit': 'Stück'}]"
2,2,32,amarettini mandelkeksezerstoßen,2 esslöffel,"{""anzahl"": 2, ""einheit"": ""essl\u00f6ffel""}","{""zutat"": ""Amarettini"", ""eigenschaft"": ""zersto...",0.03,liter,NaN,"[{'ingredient': 'Amarettini', 'properties': ['...","[{'amount': 2, 'unit': 'Esslöffel'}]"
3,3,33,deko schirmchen,2 stk.,"{""anzahl"": 2, ""einheit"": ""stk.""}","{""zutat"": ""Schirmchen"", ""eigenschaft"": ""Deko""}",2.0,stück,NaN,[],"[{'amount': '2', 'unit': 'Stück'}]"
4,4,40,chilisoße sambal oelek,0.5 tl,"{""volumen"": 0.5, ""einheit"": ""tl""}","{""zutat"": ""Chiliso\u00dfe"", ""eigenschaft"": ""Sa...",0.0025,liter,NaN,"[{'ingredient': 'Sambal Oelek', 'properties': ...","[{'amount': 0.5, 'unit': 'teelöffel'}]"
5,5,46,orangenabrieb,1 teelöffel,"{""volumen"": ""1"", ""einheit"": ""teel\u00f6ffel""}","{""zutat"": ""Teel\u00f6ffel"", ""eigenschaft"": ""or...",0.005,liter,NaN,"[{'ingredient': 'Orange', 'properties': ['Abri...","[{'amount': 1, 'unit': 'teelöffel'}]"
6,6,52,datteltomaten,4 stk.,"{""anzahl"": 4, ""einheit"": ""stk.""}","{""zutat"": ""Datteltomate"", ""eigenschaft"": ""ganz""}",4.0,stück,NaN,"[{'ingredient': 'Tomate', 'properties': ['Datt...","[{'amount': '4', 'unit': 'Stück'}]"
7,7,65,riesling trocken,100 ml,"{""volumen"": 100, ""einheit"": ""ml""}","{""zutat"": ""Riesling"", ""eigenschaft"": ""trocken""}",0.1,liter,NaN,"[{'ingredient': 'Wein', 'properties': ['trocke...","[{'volume': 100, 'unit': 'ml'}]"
8,8,103,schinkenwürfel,250 g,"{""gewicht"": 250, ""einheit"": ""g""}","{""zutat"": ""Schinkenw\u00fcrfel""}",250.0,g,NaN,"[{'ingredient': 'Schinken', 'properties': ['ge...","[{'weight': 250, 'unit': 'g'}]"
9,9,127,asiatische fischsoße,1 el,"{""volumen"": ""1"", ""einheit"": ""el""}","{""zutat"": ""Fischso\u00dfe"", ""eigenschaft"": ""as...",0.015,liter,NaN,"[{'ingredient': 'Fischsauce', 'properties': ['...","[{'amount': 1, 'unit': 'Esslöffel'}]"


In [8]:
from loguru import logger

logger.add("API_Errors.log", level = "INFO", format = "{time}:{level}:{message}")

2

In [9]:
url = "https://smarthome.uni-regensburg.de/naehrwertrechner/api/1.0/recipe_info_optifast"

def get_nut(prompt: str) -> dict | None:
    payload = { "recipe": prompt }
    headers = {"content-type": "application/json"}
    response = requests.post(url, json = payload, headers = headers, verify=False)

    if response.ok:
        return response.json()
    else:
        return None

In [10]:

def is_invalid(res: dict) -> bool:
    """
        check if the response from the API is valid or not.
        If the output is "Nicht Erkannt" then this function would returns True
        And log the error in a seperate file when an error occurs at the API
    """
    if not res or not res["detailed_info"] or res["detailed_info"][0][0]["erkannteZutat"] == "Nicht erkannt":
        return True
    return False

In [42]:
def calc_nutrition_using_annotations_generated_by_prompt1(row: pd.Series) -> str | None:
    try:
        decoded = json.loads(row["ingr_json"].replace('\'', '\"'))
        first_json = decoded[0]
        ingr = first_json["ingredient"]
    except Exception as e:
        # no json was produced, or some error occured.
        return None

    amount, unit  = float(row["norm_value"]), row["norm_unit"]

    if amount.is_integer(): amount = int(amount)

    prompt = f"{amount} {unit} {ingr}"

    res = get_nut(prompt)

    if is_invalid(res):
        logger.error(f'Error prompt: {prompt}')
        return np.nan

    try:
        return json.dumps(res)
    except Exception as e:
        return np.nan

In [43]:
data["nutrition"] = data.apply(
    lambda x: calc_nutrition_using_annotations_generated_by_prompt1(x),
    axis=1
)

2025-11-10 17:08:53.249 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt1:19 - Error prompt: 4 stück Hähnchenschlegel
2025-11-10 17:08:53.454 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt1:19 - Error prompt: 1 stück Schokoriegel
2025-11-10 17:08:53.761 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt1:19 - Error prompt: 0.03 liter Amarettini
2025-11-10 17:08:55.296 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt1:19 - Error prompt: 200 g Bratwurstbrät
2025-11-10 17:08:55.828 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt1:19 - Error prompt: 0.06 liter Salsa-Sauce
2025-11-10 17:08:56.339 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt1:19 - Error prompt: 0.02 liter Sojasoße
2025-11-10 17:08:56.478 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt1:19 - Error prompt: 0.0001 liter Agavensirup
2025-11-10 

In [44]:
# calculate the number of errors for prompt 1
data['nutrition'].isna().sum()

np.int64(42)

In [47]:
# output the errors in another file, so we can use prompt 2 on them
failed_extracts_prompt1 = data[ data['nutrition'].isna() ]
failed_extracts_prompt1.to_csv("failed_extracts_prompt1.csv", index=False)

In [48]:
# now let's use prompt 2 on the failed extracts from prompt 1 so we can get a total coverage of both prompts

data = pd.read_csv("failed_extracts_prompt1.csv", dtype=str)
data.head(10)

,Unnamed: 0.1,Unnamed: 0,ingredient,amount,ingr_annotation,amount_annotation,norm_value,norm_unit,nutrition,ingr_json,amount_json
0,0,15,bio hähnchenschlegel,4 stk.,"{""anzahl"": 4, ""einheit"": ""stk.""}","{""zutat"": ""H\u00e4hnchenschlegel"", ""eigenschaf...",4.0,stück,NaN,"[{'ingredient': 'Hähnchenschlegel', 'propertie...","[{'amount': 4, 'unit': 'Stück'}]"
1,1,31,schokoriegel,1 stk.,"{""anzahl"": 1, ""einheit"": ""stk.""}","{""zutat"": ""Schokoriegel""}",1.0,stück,NaN,[{'ingredient': 'Schokoriegel'}],"[{'amount': 1, 'unit': 'Stück'}]"
2,2,32,amarettini mandelkeksezerstoßen,2 esslöffel,"{""anzahl"": 2, ""einheit"": ""essl\u00f6ffel""}","{""zutat"": ""Amarettini"", ""eigenschaft"": ""zersto...",0.03,liter,NaN,"[{'ingredient': 'Amarettini', 'properties': ['...","[{'amount': 2, 'unit': 'Esslöffel'}]"
3,3,33,deko schirmchen,2 stk.,"{""anzahl"": 2, ""einheit"": ""stk.""}","{""zutat"": ""Schirmchen"", ""eigenschaft"": ""Deko""}",2.0,stück,NaN,[],"[{'amount': '2', 'unit': 'Stück'}]"
4,10,142,brunch,1 becher,"{""volumen"": 1, ""einheit"": ""becher""}","{""zutat"": ""Becher"", ""eigenschaft"": ""Brunch""}",0.25,liter,NaN,[],"[{'amount': 1, 'unit': 'Becher'}]"
5,13,181,bratwurstbrät,200 g,"{""gewicht"": 200, ""einheit"": ""g""}","{""zutat"": ""Bratwurstbr\u00e4t""}",200.0,g,NaN,"[{'ingredient': 'Bratwurstbrät', 'properties':...","[{'weight': 200, 'unit': 'g'}]"
6,17,279,salsa-sauce,4 el,"{""volumen"": 4, ""einheit"": ""el""}","{""zutat"": ""Salsa-Sauce""}",0.06,liter,NaN,"[{'ingredient': 'Salsa-Sauce', 'properties': [...","[{'amount': 4, 'unit': 'esslöffel'}]"
7,20,309,sojasoße dunkel,1 schuss,"{""volumen"": 1, ""einheit"": ""Schuss""}","{""zutat"": ""So\u00dfe"", ""eigenschaft"": ""dunkel""}",0.02,liter,NaN,"[{'ingredient': 'Sojasoße', 'properties': ['du...","[{'amount': 1, 'unit': 'Schuss'}]"
8,21,317,agavensirup,2 tropfen,"{""anzahl"": 2, ""einheit"": ""tropfen""}","{""zutat"": ""Tropfen"", ""eigenschaft"": ""Agavensir...",0.0001,liter,NaN,"[{'ingredient': 'Agavensirup', 'properties': []}]","[{'amount': '2', 'unit': 'Tropfen'}]"
9,22,331,mangoldblätter rot,50 g,"{""gewicht"": 50, ""einheit"": ""g""}","{""zutat"": ""Mangoldblatt"", ""eigenschaft"": ""rot""}",50.0,g,NaN,"[{'ingredient': 'Mangoldblatt', 'properties': ...","[{'weight': 50, 'unit': 'g'}]"


In [49]:
from ollama import Client
llama_3_3 = Client(host="http://intern.schlaubox.de:11434", timeout=500)
from pathlib import Path

try:
    HERE = Path(__file__).parent
except Exception as e:
    HERE = Path.cwd()

## PROMPT 2

# another approach to extract ingredients

def extract_ingredients_json(ingr_txt: str) -> str:
    messages = [
        { "role" : "user",
          "content":
              f"""
                    Your task is to identify cooking ingredients in texts and output them in
                    the nominative singular. If characteristics such as color,  condition,
                    processing, quality, or origin are mentioned for an ingredient, ignore them.

                    You must output as single word, which should be the ingredient.
                    DO not output
                    anything else. You must always output a valid ingredient.

                    Please carefully observe these examples:

                        input: schokoriegel
                        output: riegel

                        input: amarettini mandelkeksezerstoßen
                        output: mandelkekse

                        input: deko schirmchen
                        output: schirmchen

                        input: asiatische fischsoße
                        output: fischsoße

                        input: bisquits rund
                        output: bisquits

                        input: agavensirup
                        output: sirup

                        input: prinzessbohnen gewürfelt
                        output: bohnen

                        input: wurzelgemüse
                        output: gemüse

                        input: jostabeeren
                        output: beeren

                        input: curcuma gelbwurz
                        output: curcuma

                        input: pankomehl
                        output: mehl

                        input: stevia flüssig
                        output: stevia

                        input: weißkohl frisch
                        output: kohl

                        input: stevia flüssig
                        output: stevia

                        input: speiseöl
                        output: öl

                    Now identify the ingredients in this text: {ingr_txt}
                """
        }
    ]

    llm_analysis = llama_3_3.chat(model='llama3.3:70b', messages=messages)
    answer = llm_analysis['message']['content']

    return answer

In [50]:
## test this file using the testing.ipynb file and see if we manage to reduce the errors
data['ingr_identified'] = data.apply(
    lambda x: extract_ingredients_json(x["ingredient"]),
    axis = 1,
)

data.to_csv("gemma_annotated_prompt2.csv", index=False)

In [52]:
def calc_nutrition_using_annotations_generated_by_prompt2(x: pd.Series) -> str:
    """
        Calculate the nutrition using annotations generated by prompt 2 over data that prompt 1 failed
    """
    amount, unit  = float(x["norm_value"]), x["norm_unit"]

    if amount.is_integer(): amount = int(amount)

    ingr = x["ingr_identified"]

    prompt = f"{amount} {unit} {ingr}"

    res = get_nut(prompt)

    if is_invalid(res):
        logger.error(prompt)
        return np.nan

    try:
        return json.dumps(res)
    except Exception as e:
        return np.nan

In [53]:
data['nutrition'] = data.apply(
    lambda x: calc_nutrition_using_annotations_generated_by_prompt2(x),
    axis = 1,
)

2025-11-10 17:24:15.667 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt2:16 - 4 stück schlegel
2025-11-10 17:24:16.075 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt2:16 - 0.03 liter mandelkekse
2025-11-10 17:24:16.281 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt2:16 - 2 stück schirmchen
2025-11-10 17:24:16.486 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt2:16 - 0.25 liter brunch
2025-11-10 17:24:17.408 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt2:16 - 0.02 liter soße
2025-11-10 17:24:18.021 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt2:16 - 50 g mangoldblätter -> blätter 

The ingredient is: blätter
2025-11-10 17:24:19.048 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prompt2:16 - 0.03 liter soße
2025-11-10 17:24:28.262 | ERROR    | __main__:calc_nutrition_using_annotations_generated_by_prom

In [54]:
# calculate the number of errors for prompt 2
data["nutrition"].isna().sum()

np.int64(18)

In [56]:
# Filter out rows where nutrition is None
filtered = data[data['nutrition'].isna()]

# This is the final failed extracts, we cannot do any better with these prompts. Part of the reason is that
# many of these are unsolvabel. Some of them are either not ingredients or the ingredient is not supported
# by the API

filtered.to_csv( HERE.parent / "failed_extracts_final.csv", index=False)
filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18 entries, 0 to 39
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0.1       18 non-null     object
 1   Unnamed: 0         18 non-null     object
 2   ingredient         18 non-null     object
 3   amount             18 non-null     object
 4   ingr_annotation    18 non-null     object
 5   amount_annotation  18 non-null     object
 6   norm_value         18 non-null     object
 7   norm_unit          18 non-null     object
 8   nutrition          0 non-null      object
 9   ingr_json          18 non-null     object
 10  amount_json        18 non-null     object
 11  ingr_identified    18 non-null     object
dtypes: object(12)
memory usage: 1.8+ KB
